In [5]:
import json
from typing import *
def read_json(path: str) -> List[Dict]:
    with open(path, 'r', encoding='utf-8') as file:
        return json.loads(file.read())


In [4]:
def read_jsonl(path: str) -> List[Dict]:
    with open(path, 'r', encoding='utf-8') as file:
        data = []
        lines = file.readlines()
        for line in lines:
            data.append(json.loads(line))
    return data

In [27]:
label2id = {"因果": 0, "时序": 1, "无": 2}

In [28]:
id2label = {v: k for k, v in label2id.items()} 

In [6]:
traindata = read_jsonl(path="dataset/train.json")
len(traindata)

8000

In [7]:
testdata = read_jsonl(path = "dataset/testa.json")
len(testdata)

1000

In [9]:
sample = traindata[0]
sample

{'new-ID': '10290',
 'doc': '成都市原市长助理门生涉嫌受贿案侦查终结，移送审查起诉_一号专案_澎湃新闻-ThePape。\n\n中新网成都1月5日电，5日，成都市人民检察院官方网站公布了近段时间成都市检察机关查处的4起职务犯罪案件信息。其中，原成都市市长助理、秘书长门生（副厅级）涉嫌受贿一案，由成都市人民检察院侦查终结，移送审查起诉。门生，男，汉族，1959年3月生，四川内江人。2009年门生因抗震救灾有功等原因，兼任成都市市长助理。2013年2月，门生转任成都市市长助理、市政府秘书长，并兼市政府办公厅党组书记，市委党校（行政学院）第一副校（院）长等职迄今。2015年06月10日，成都市纪委发布消息称，经成都市委同意，成都市纪委对成都市市长助理、市政府秘书长门生涉嫌严重违纪违法问题进行立案调查。门生是2014年8月以来，成都继陈争鸣、刘俊林及周鸿德之后落马的第4位市长助理。2015年9月17日，成都市人民检察院依法决定对成都市政府原市长助理、秘书长门生（副厅级）受贿立案侦查。2015年9月25日，经四川省人民检察院决定，依法对门生予以逮捕。此外，成都市检察院还公布了成都武侯区投资促进局副局长岳文婷、成都锦江区三圣街道办事处办公室原主任胡杨、成都市公积金管理中心原党委书记胡旭光的职务犯罪案件信息。',
 'events': [{'id': '10290_1',
   'event-information': {'trigger': [{'text': '立案侦查',
      'start': 422,
      'end': 426}],
    'event_type': '司法行为_立案调查'}},
  {'id': '10290_2',
   'event-information': {'trigger': [{'text': '逮捕', 'start': 457, 'end': 459}],
    'event_type': '司法行为_拘捕'}}],
 'relations': [{'one_event_id': '10290_1',
   'other_event_id': '10290_2',
   'relation_type': '时序'}]}

In [10]:
sample["doc"][422:426]

'立案侦查'

In [16]:
event_types = []
for d in traindata:
    for evt in d["events"]:
        # evt_type = evt["event-information"]["event_type"].split("_")[0]
        evt_type = evt["event-information"]["event_type"]
        if evt_type not in event_types:
            event_types.append(evt_type)
print(event_types)

['司法行为_立案调查', '司法行为_拘捕', '突发事件_社会安全_经济安全（抢劫、盗窃等）', '突发事件_社会安全_袭击', '人生_死亡', '突发事件_社会安全_爆炸', '突发事件_事故灾难_交通运输事故', '司法行为_罚款', '司法行为_起诉', '突发事件_社会安全_罢工', '财经/交易_出售', '突发事件_事故灾难_火灾', '突发事件_自然灾害_气象灾害', '组织关系_解雇', '突发事件_事故灾难_公共设施事故', '突发事件_事故灾难_生态污染事件', '突发事件_社会安全_劫持', '人生_生病/检查', '突发事件_事故灾难_工矿商贸安全事件', '竞赛行为_禁赛', '司法行为_举报', '突发事件_公共卫生_疫情', '突发事件_自然灾害_地震灾害', '突发事件_自然灾害_水旱灾害', '财经/交易_降价', '突发事件_自然灾害_地质灾害', '财经/交易_涨价', '交往行为_威胁', '突发事件_社会安全_游行聚集', '产品行为_检验', '突发事件_公共卫生_食品安全', '人生_产子', '组织行为_开幕', '组织行为_闭幕', '产品行为_下架', '突发事件_自然灾害_生物灾害', '移动_运输', '司法行为_出狱', '突发事件_社会安全_民族宗教', '司法行为_入狱', '移动_出行', '组织关系_选举', '产品行为_召回', '交往行为_道歉', '竞赛行为_竞赛', '交往行为_见面', '产品行为_制造', '交往行为_联系', '人生_结婚', '人生_离婚', '财经/交易_股票下跌', '财经/交易_股票上涨', '人生_获奖', '产品行为_发布', '竞赛行为_晋级', '交往行为_感谢', '组织关系_离职', '财经/交易_上市', '财经/交易_收购', '组织关系_裁员', '人生_升学']


In [17]:
len(event_types)

61

In [25]:
path = "/data/jupyter/root/disk/0_BERTmodels/huggingface/bert-base-chinese"
from transformers import AutoModel, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModel.from_pretrained(path)

Some weights of the model checkpoint at /data/jupyter/root/disk/0_BERTmodels/huggingface/bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at /data/jupyter/root/disk/0_BERT

In [26]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(21128, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (history_embeddings): Embedding(13, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropo

In [29]:
sample

{'new-ID': '10290',
 'doc': '成都市原市长助理门生涉嫌受贿案侦查终结，移送审查起诉_一号专案_澎湃新闻-ThePape。\n\n中新网成都1月5日电，5日，成都市人民检察院官方网站公布了近段时间成都市检察机关查处的4起职务犯罪案件信息。其中，原成都市市长助理、秘书长门生（副厅级）涉嫌受贿一案，由成都市人民检察院侦查终结，移送审查起诉。门生，男，汉族，1959年3月生，四川内江人。2009年门生因抗震救灾有功等原因，兼任成都市市长助理。2013年2月，门生转任成都市市长助理、市政府秘书长，并兼市政府办公厅党组书记，市委党校（行政学院）第一副校（院）长等职迄今。2015年06月10日，成都市纪委发布消息称，经成都市委同意，成都市纪委对成都市市长助理、市政府秘书长门生涉嫌严重违纪违法问题进行立案调查。门生是2014年8月以来，成都继陈争鸣、刘俊林及周鸿德之后落马的第4位市长助理。2015年9月17日，成都市人民检察院依法决定对成都市政府原市长助理、秘书长门生（副厅级）受贿立案侦查。2015年9月25日，经四川省人民检察院决定，依法对门生予以逮捕。此外，成都市检察院还公布了成都武侯区投资促进局副局长岳文婷、成都锦江区三圣街道办事处办公室原主任胡杨、成都市公积金管理中心原党委书记胡旭光的职务犯罪案件信息。',
 'events': [{'id': '10290_1',
   'event-information': {'trigger': [{'text': '立案侦查',
      'start': 422,
      'end': 426}],
    'event_type': '司法行为_立案调查'}},
  {'id': '10290_2',
   'event-information': {'trigger': [{'text': '逮捕', 'start': 457, 'end': 459}],
    'event_type': '司法行为_拘捕'}}],
 'relations': [{'one_event_id': '10290_1',
   'other_event_id': '10290_2',
   'relation_type': '时序'}]}

In [31]:
events = sample["events"]
trigger1 = events[0]["event-information"]["trigger"][0]
trigger2 = events[1]["event-information"]["trigger"][0]

In [33]:
text1 = sample["doc"][(trigger1["start"]-10):trigger1["end"]+10)]
text2 = sample["doc"][(trigger1["start"]-10):trigger1["end"]+10)]

'成都市原市长助理门生涉嫌受贿案侦查终结，移送审查起诉_一号专案_澎湃新闻-ThePape。\n\n中新网成都1月5日电，5日，成都市人民检察院官方网站公布了近段时间成都市检察机关查处的4起职务犯罪案件信息。其中，原成都市市长助理、秘书长门生（副厅级）涉嫌受贿一案，由成都市人民检察院侦查终结，移送审查起诉。门生，男，汉族，1959年3月生，四川内江人。2009年门生因抗震救灾有功等原因，兼任成都市市长助理。2013年2月，门生转任成都市市长助理、市政府秘书长，并兼市政府办公厅党组书记，市委党校（行政学院）第一副校（院）长等职迄今。2015年06月10日，成都市纪委发布消息称，经成都市委同意，成都市纪委对成都市市长助理、市政府秘书长门生涉嫌严重违纪违法问题进行立案调查。门生是2014年8月以来，成都继陈争鸣、刘俊林及周鸿德之后落马的第4位市长助理。2015年9月17日，成都市人民检察院依法决定对成都市政府原市长助理、秘书长门生（副厅级）受贿立案侦查。2015年9月25日，经四川省人民检察院决定，依法对门生予以逮捕。此外，成都市检察院还公布了成都武侯区投资促进局副局长岳文婷、成都锦江区三圣街道办事处办公室原主任胡杨、成都市公积金管理中心原党委书记胡旭光的职务犯罪案件信息。'